# Hands-on genomics with Hail

[Hail](https://hail.is/) is an open-source library developed by the Broad to learn from the huge amounts of genomic data being generated. This notebook walks through some of the major concepts in statistical and population genetics using Hail.

In [ ]:
import hail as hl
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

## 1000 Genomes

The [1000 Genomes](http://www.internationalgenome.org/data/) dataset is the product of an international effort to sequence a diverse set of individuals and make the resulting data public for both research and educational use.

We use a small chunk of this public 1000 Genomes dataset, created by randomly selecting a small fraction of the genotyped SNPs, and using about 300 samples from the full 2500.

In [ ]:
hl.utils.get_1kg('data/')

In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')

## Getting to know genomic data

Visualizing data is very important! We can see the data in its matrix form here.

In [ ]:
mt.GT.show()

## Importance of phenotype information

"Phenotype" refers to a characteristic of an individual resulting from genetics and environment. Genomic data might be interesting on its own, but combined with information about health and disease status, genomic data can help us better understand the basis of human disease.

We will import a separate source of simulated phenotype information now.

In [ ]:
table = hl.import_table('data/1kg_annotations.txt', impute=True, key='Sample').cache()

In [ ]:
table.show()

In [ ]:
mt = mt.annotate_cols(pheno = table[mt.s])

## Understanding our SNPs

Recall that there are four bases: A, T, G, C.  We can count the number of each of the 12 kinds of SNP mutations in our dataset:

In [ ]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.delimit(mt.alleles, '/')))
pprint(snp_counts)

In [ ]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

## Quality Control

Sequencing data is noisy and probabilistic. It's important to perform rigorous quality control so that technical artifacts from the sequencing process do not leak into published results!

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
show(hl.plot.histogram(mt.variant_qc.call_rate))

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

In [ ]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

## Genome-Wide Association Study (GWAS)

A GWAS is the composition of many millions of independent tests, which interrogate whether each the genotypes at each variant are correlated with the trait of interest. In Hail, this is called "linear_regression_rows" to indicate that we are doing a separate linear regression for each row of the genotype matrix.

In [ ]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption, 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0])

## Visualizing GWAS results

Hail makes it easy to visualize results! Let's make a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot):

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is linked to caffeine consumption (unlikely), or there's a confounder (much more likely)!

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. How do we do that? PCA!

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

In [ ]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

## Human history through principal components

Useful link: [population and super-population codes in 1000 Genomes](http://www.internationalgenome.org/category/population/).

In [ ]:
p = hl.plot.scatter(mt.scores[0], 
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2',
                    hover_fields={'ID': mt.s, 'pop2': mt.pheno.Population},
                    size=7)
show(p)

In [ ]:
p = hl.plot.scatter(mt.scores[2], 
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC3', ylabel='PC2',
                    hover_fields={'ID': mt.s, 'pop2': mt.pheno.Population},
                    size=7)
show(p)

In [ ]:
p = hl.plot.scatter(mt.scores[2], 
                    mt.scores[3],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC3', ylabel='PC4',
                    hover_fields={'ID': mt.s, 'pop2': mt.pheno.Population},
                    size=7)
show(p)

## Running an appropriately-controlled GWAS

Now we can rerun our linear regression, controlling for sex and the first few principal components.

In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption, 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2], mt.scores[3]])

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)